In [41]:
import urllib.request
import zipfile
import os
import numpy as np
import pandas as pd
import re
import requests

`download_geonames_data`

Deze functie automatiseert het proces van het downloaden van de data over steden wereldwijd via GeoNames. Deze data is ingepakt als ZIP-bestand. Na het uitpakken van de data wordt  het gedownloade bestand verwijderd.

In [42]:
def download_geonames_data(url):

    filename = os.path.basename(url)
    urllib.request.urlretrieve(url, filename)

    with zipfile.ZipFile(filename, 'r') as zipRef:
        zipRef.extractall()

    os.remove(filename)

`combine_geonames_data`

Deze functie dient ter opschoning van de GeoNames-data. Eerst worden namen gegeven aan de kolommen zodat deze beter op te roepen zijn in het programma. Voor elke rij wordt gekekeken of de landcode `NL` (Nederland) of `BE` (België) is. Rijen die niet aan deze voorwaarde voldoen worden gefilterd. Vervolgens wordt de informatie over alternatieve plaatsnamen opgeschoond:
1. Alle tekens die geen standaard ASCII-teken zijn, worden verwijderd;
2. Missende waarden (`np.nan`) worden vervangen door een lege waarde (`''`);
3. Data die staan opgeslagen als `list` (lijst met gegevens) worden omgezet naar één reeks tekens;
4. Achtereenvolgende en overtollige komma's worden verwijderd.

Ten slotte worden onnodige kolommen uit de tabel verwijderd.

In [43]:
def combine_geonames_data(folder, city_size):
    columns = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date']
    filename = f'{folder}/data/cities{city_size}.txt'

    dataframe = pd.read_csv(filename, names = columns, sep = '\t', low_memory = False)
    dataframe = dataframe[dataframe['country code'].isin(['NL', 'BE'])]	

    dataframe['alternatenames'] = dataframe['alternatenames'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x) if isinstance(x, str) else x)
    dataframe['alternatenames'] = dataframe['alternatenames'].replace(np.nan, '')
    dataframe['alternatenames'] = dataframe['alternatenames'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    dataframe['alternatenames'] = dataframe['alternatenames'].str.replace(r',+', ',', regex = True).str.strip(',')

    dataframe['city size'] = city_size
    
    required_columns = ['geonameid', 'name', 'alternatenames', 'latitude', 'longitude', 'country code', 'city size']
    dataframe = dataframe[required_columns]

    return dataframe

`import_geonames_data`

In de betreffende map `folder` wordt gekeken naar de GeoNames data op basis van het inwonersaantal. Hierop wordt de bovenstaande functie `combine_geonames_data` toegepast om de data in één tabel zichtbaar te maken. 

Omdat er overlap kan ontstaan tussen de steden op basis van hun grootte (steden met minder dan 5.000 inwoners hebben óók minder dan 500 inwoners in deze tabel) worden rijen die dezelfde `geonameid` hebben verwijderd. 

Als de data is samengevoegd worden de losse GeoNames-tabellen verwijderd.

In [44]:
def import_geonames_data(folder):
    geonames_dataframe = []
    city_sizes = ['15000', '5000', '1000', '500']

    for city_size in city_sizes:
        dataframe = combine_geonames_data(folder, city_size)
        geonames_dataframe.append(dataframe)
    
    geonames_data = pd.concat(geonames_dataframe, ignore_index = True)
    geonames_data.drop_duplicates(subset = 'geonameid', keep = 'first', inplace = True)

    for city_size in city_sizes:
        filename = f'{folder}/data/cities{city_size}.txt'
        os.remove(filename)

    return geonames_data


`find_whg_variants`

Via de `API` van World Historical Gazetteer (WHG) wordt aanvullende data over alternatieve stadsnamen opgezocht. Dit geschiedt op basis van de data over de stadsnaam (`city` in de kolom) en de landcode (`country_code` in de kolom). Dit levert JSON-data op, waarvan de data over `variants` wordt gefilterd. Deze data wordt opgeslagen in `results`, elke keer dat deze functie wordt aangeroepen.

In [45]:
def find_whg_variants(city, country_code):
    url = f"https://whgazetteer.org/api/index/?name={city}&ccode={country_code}"

    response = requests.get(url)
    data = response.json()
    features = data['features']
    results = []

    for feature in features:
        properties = feature['properties']
        variants = properties['variants']

        for variant in variants:
            if re.match('^[a-zA-Z/s]+$', variant):
                results.append(variant)

        results.append(properties['title'])

    return results

`find_variants`

Deze code maakt gebruik van de `find_whg_variants` functie. Voor elke rij in de `geonames_data`-tabel, wordt informatie over de `city`, `country_code` en `alternatenames` verzameld. Vervolgens wordt op basis van de stadsnaam en de landcode gezocht naar WHG-varianten. De resultaten worden opgeslagen las `whg_variants`.

De resultaten worden samengevoegd met de bestaande data onder de naam `whg_geonames_variants`. Indien er aanvullende informatie beschikbaar was in de WHG-database wordt deze toegevoegd aan de GeoNames-tabel. Anders wordt alleen de stadsnaam toegevoegd als alternatieve stadsnaam.

In [46]:
def find_variants(geonames_data):
    for index, row in geonames_data.iterrows():
        city = row['name']
        country_code = row['country code']
        variants = row['alternatenames'].split(',') if isinstance(row['alternatenames'], str) else []
        whg_variants = find_whg_variants(city, country_code)

        whg_geonames_variants = [city] + variants + whg_variants
        whg_geonames_variants = [item for item in whg_geonames_variants if len(item) > 1]
        whg_geonames_variants = list(set(whg_geonames_variants))

        geonames_data.at[index, 'alternatenames'] = whg_geonames_variants if whg_geonames_variants else [city]

    return geonames_data

``main``

Als de code al eens heeft gedraaid, en de GeoNames data is opgeslagen op de computer, wordt de code niet uitgevoerd. Anders wordt via de downloadlink van GeoNames de data over steden van meer dan 15.000, minder dan 5.000, 1.000 en 500 inwoners verzameld. 

Vervolgens wordt de data van GeoNames en WHG gekoppeld volgens bovenstaande code

De output van het programma is een bestand genaamd `geonames_data.csv`

In [47]:
def main():
    if os.path.isfile('geonames_data.csv'):
        geonames_data = pd.read_csv('geonames_data.csv')
        return geonames_data
    
    else:  
        urls = [f'https://download.geonames.org/export/dump/cities{i}.zip' for i in [15000, 5000, 1000, 500]]

        for url in urls:
            download_geonames_data(url)  
        
        folder = os.path.dirname(os.getcwd())

        geonames_data = find_variants(import_geonames_data(folder))
        geonames_data.to_csv(f'{folder}/data/geonames_data.csv', index=False)

        return geonames_data

In [48]:
if __name__ == '__main__':
    main()